In [2]:
import cv2
import numpy as np

from glob import glob

chars_list = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
chars_dict = {c: chars_list.index(c) for c in chars_list}

char_img_paths = glob('data/chars/*.jpg')
char_img_paths[:5]


np_images = []
raw_labels = []

for char_img_path in char_img_paths:
    np_image = cv2.imread(char_img_path, cv2.IMREAD_GRAYSCALE)
    raw_label = char_img_path.split('/')[-1].split('-')[0]
    
    np_images.append(np_image)
    raw_labels.append(chars_dict[raw_label])
np_images = np.array(np_images) / 255.0
raw_labels = np.array(raw_labels)  

In [4]:
np_images.shape, raw_labels.shape, len(chars_list)

((5056, 36, 30), (5056,), 36)

In [11]:
np_images = np_images.reshape(-1, 36, 30, 1)

In [12]:
from tensorflow.python.keras._impl.keras.utils.np_utils import to_categorical # convert to one-hot-encoding
vec_labels = to_categorical(raw_labels, num_classes=36)
vec_labels

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = \
    train_test_split(np_images, vec_labels, test_size=0.1)

In [27]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.keras._impl.keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.python.keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
training = tf.placeholder_with_default(False, shape=(), name='training')

def random_batch(x_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(x_train), batch_size)
    x_batch = x_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return x_batch, y_batch

model = Sequential()
model.add(Conv2D(32, 5, padding='same', activation='relu', input_shape=(36, 30, 1)))
model.add(Conv2D(32, 5, padding='same', activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.25))
model.add(Conv2D(64, 3, padding='Same', activation='relu'))
model.add(Conv2D(64,3, padding='Same', activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax'))
optimizer = Adam()
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 20 
batch_size = 80

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val,y_val),
                              verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/20
1s - loss: 2.8741 - acc: 0.2078 - val_loss: 0.6890 - val_acc: 0.8419
Epoch 2/20
0s - loss: 0.9416 - acc: 0.7059 - val_loss: 0.1417 - val_acc: 0.9723
Epoch 3/20
0s - loss: 0.4378 - acc: 0.8610 - val_loss: 0.0762 - val_acc: 0.9822
Epoch 4/20
0s - loss: 0.2500 - acc: 0.9148 - val_loss: 0.0669 - val_acc: 0.9921
Epoch 5/20
0s - loss: 0.1937 - acc: 0.9380 - val_loss: 0.0511 - val_acc: 0.9941
Epoch 6/20
0s - loss: 0.1655 - acc: 0.9489 - val_loss: 0.0535 - val_acc: 0.9921
Epoch 7/20
0s - loss: 0.1341 - acc: 0.9570 - val_loss: 0.0487 - val_acc: 0.9941
Epoch 8/20
0s - loss: 0.1275 - acc: 0.9625 - val_loss: 0.0397 - val_acc: 0.9941
Epoch 9/20

Epoch 00008: reducing learning rate to 0.0005000000237487257.
0s - loss: 0.1156 - acc: 0.9628 - val_loss: 0.0444 - val_acc: 0.9941
Epoch 10/20
0s - loss: 0.0828 - acc: 0.9749 - val_loss: 0.0433 - val_acc: 0.9941
Epoch 11/20
0s - loss: 0.0665 - acc: 0.9792 - val_loss: 0.0384 - val_acc: 0.9941
Epoch 12/20

Epoch 00011: reducing learning rate to 0.0